In [87]:
import pdb
import pandas as pd
import numpy as np
import os
np.random.seed(3101)
random_state = np.random.RandomState(3101)

# Importing data:
bank1 = archive  
bank2 = bank+marketing

In [88]:
# print(os.getcwd())
bank_df_train = pd.read_csv('./data/archive3/train.csv', header=True)
bank_df_test = pd.read_csv('./data/archive3/test.csv')

TypeError: Passing a bool to header is invalid. Use header=None for no header or header=int or list-like of ints to specify the row(s) making up the column names

In [ ]:
print(bank_df_train.head())
print(len(bank_df_train))

bank_df_train_clean = bank_df_train.drop_duplicates(subset="CustomerId", keep="first")
print(len(bank_df_train_clean))

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  
0       0  
1       0  
2       0  
3       0  
4 

In [ ]:
print(bank_df_test.head())
len(bank_df_test)
duplicates = bank_df_test.duplicated(subset=['CustomerId'], keep=False)
print(duplicates)
#no "exited column"

       id  CustomerId    Surname  CreditScore Geography  Gender   Age  Tenure  \
0  165034    15773898   Lucchese          586    France  Female  23.0       2   
1  165035    15782418       Nott          683    France  Female  46.0       2   
2  165036    15807120         K?          656    France  Female  34.0       7   
3  165037    15808905  O'Donnell          681    France    Male  36.0       8   
4  165038    15607314    Higgins          752   Germany    Male  38.0      10   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
0       0.00              2        0.0             1.0        160976.75  
1       0.00              1        1.0             0.0         72549.27  
2       0.00              2        1.0             0.0        138882.09  
3       0.00              1        1.0             0.0        113931.57  
4  121263.62              1        1.0             0.0        139431.00  
0          True
1          True
2          True
3          True
4    

# Adding and populating features

### 14. Customer Service Calls: Higher calls might indicate issues and dissatisfaction, affecting churn. 
modelling with poisson distribution


In [ ]:
from scipy.stats import poisson

avg_call_rate_per_year = 12
bank_df_train_clean['NumCalls'] = np.random.poisson(avg_call_rate_per_year, size = len(bank_df_train_clean))
print(bank_df_train_clean.head())

(bank_df_train_clean["NumCalls"]<0).any() #no negative calls

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  NumCalls  
0       0        12  
1       0        1

C:\Users\caboo\AppData\Local\Temp\ipykernel_34220\3054993029.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df_train_clean['NumCalls'] = np.random.poisson(avg_call_rate_per_year, size = len(bank_df_train_clean))


True

### 15. Total Relationship Count: Reflects the breadth of the customer's relationship with the bank. 
There are two main ways banks interpret total relationship count:

Number of Products and Services: This refers to the variety of products and services a customer uses from the bank. For example, if a customer has a checking account, savings account, and credit card with the same bank, their total relationship count based on products would be 3. A higher number indicates a broader and potentially more complex relationship. -> (sum of NumOfProducts and HasCrCard)

Total Assets Under Management: This focuses on the total value a customer entrusts to the bank. It considers the sum of their deposits (checking, savings), investments (stocks, bonds), and borrowings (loans, mortgages) managed through the bank. A higher total asset value suggests a stronger and more significant relationship for the bank. -> (we have deposit(balance). will need to create investments and borrowings)

### 31. Churn Date: The exact date the customer decided to exit.


In [130]:
start_date = "2023-01-01"
end_date = "2023-12-31"

random_dates = pd.date_range(start=start_date, end=end_date, freq='D')
random_dates_str = random_dates.strftime('%Y-%m-%d')
# print(np.random.choice(random_dates_str))
bank_df_train_clean['ChurnDate'] = np.where(bank_df_train_clean['Exited'] == 1, np.random.choice(random_dates_str) , np.nan) 
# bank_df_train_clean['ChurnDate'] = np.where(bank_df_train_clean['ChurnDate'] != pd.NaT, bank_df_train_clean['ChurnDate'].dt.strftime('%Y-%m-%d'), pd.Nat)
# bank_df_train_clean['ChurnDate'] = bank_df_train_clean['ChurnDate'].dt.strftime('%Y-%m-%d')
print(bank_df_train_clean['ChurnDate'].head(10))

0           nan
1           nan
2           nan
3           nan
4           nan
5    2023-11-15
6           nan
7           nan
8           nan
9           nan
Name: ChurnDate, dtype: object


C:\Users\caboo\AppData\Local\Temp\ipykernel_34220\1686782772.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df_train_clean['ChurnDate'] = np.where(bank_df_train_clean['Exited'] == 1, np.random.choice(random_dates_str) , np.nan)


### 32. Sign Up Date: When did the customer create an account

# EDA
1. covarience matrix
2. LDA
3. apriori
4. domain knowledge

### 1. Covariance matrix

In [ ]:
#code for covariance matrix

### 2. LDA

In [ ]:
#code for LDA

### 3. Aprior

In [ ]:
#code for apriori

### 4. Domain knowledge

In [ ]:
#domain knowledge....

# Model training:
1. Random forest
2. XGBoost
3. Logistic regression